In [5]:
from behav3d.preprocessing.imaris_preprocessing import run_imaris_preprocessing
from behav3d.analysis.feature_extraction import (
    calculate_track_features,
    filter_tracks,
    summarize_track_features
)
import yaml
import pandas as pd

### Imaris preprocessing
Imaris outputs their calculated statistics in a certain .csv format that is not yet compatible with BEHAV3D processing. <br>
You therefore have to preprocess the resulting CSVs before applying BEHAV3D analysis.

Additionally, as Imaris uses surfaces that can not be extracted as voxel-based segments, the features "organoid contact", "t-cell contact" and "dead dye mean" can not be calculated outside of Imaris and thus must be calculated and extracted in Imaris

The following code preprocesses the Imaris segments based on their:
- centroid position (zyx)
- A distance to organoid channel that needs to be set up inside of Imaris and exported as an additonal channel
- The statistics of the dead dye mean, which is based on a dead dye channel

In [2]:
imaris_positions_csv = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/aim_dp1_exp13_img004_dietcart_n3_lg_car2_CD4_Position.csv"
imaris_dead_dye_mean_csv= "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/aim_dp1_exp13_img004_dietcart_n3_lg_car2_CD4_Intensity_Mean_Ch=5_Img=1.csv"
imaris_organoid_distance_csv = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/aim_dp1_exp13_img004_dietcart_n3_lg_car2_CD4_Shortest_Distance_to_Surfaces_Surfaces=Leukemia.csv"
imaris_tcell_distance_csv = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/aim_dp1_exp13_img004_dietcart_n3_lg_car2_CD4_Shortest_Distance_to_Surfaces_Surfaces=CD8.csv"


# output_path can be left empty to automatically name the output 
# and put it in the same folder as "imaris_positions_csv"
processed_imaris_output_path = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/df_imaris_preprocessed.csv"

run_imaris_preprocessing(
    df_positions_path=imaris_positions_csv,
    df_organoid_distances_path=imaris_organoid_distance_csv,
    df_tcell_distances_path=imaris_tcell_distance_csv,
    df_dead_dye_means_path=imaris_dead_dye_mean_csv,
    output_path=processed_imaris_output_path
)

### Load in config and metadata csv
Make sure to edit the metadata .csv accordingly and change the "tcell_track_csv" to the processed imaris .csv files

In [12]:
config_path = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/config.yml"

with open(config_path, "r") as parameters:
        config=yaml.load(parameters, Loader=yaml.SafeLoader)
        
# This can already be defined in the config, but this line makes sure 
# that the processing mode is set to Imaris processing
config["imaris"]=True

metadata = pd.read_csv(config["metadata_csv_path"])
for k,v in config.items():
        print(f"{k}: {v}")
metadata


output_dir: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/
metadata_csv_path: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/metadata.csv
imaris: True
ilastik_path: /Applications/ilastik-1.4.0b27-OSX.app/Contents/ilastik-release/run_ilastik.sh
ilastik_pixel_classifier_model: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_pix_clas_v5.ilp
ilastik_organoid_segmentation_model: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_obj_clas_v4.ilp
ilastik_organoid_postprocessing_model: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_faketrack_obj_split_v4.ilp
ilastik_tcell_segmentation_model: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_obj_clas_TCELL.ilp
ilastik_tcell_postprocessing_model: /Us

### Calculate the track features

In [4]:
df_tracks = calculate_track_features(config, metadata)
df_tracks_filt = filter_tracks(df_tracks, config, metadata)
df_tracks_summ = summarize_track_features(df_tracks_filt, config)

--------------- Processing tcells: AIM_MB2_Exp58_Img003_donor899_MA quantified ---------------


KeyError: 'contact_threshold'